In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# TODO: preprocess and compute the TF-IDF of this database
def preprocessing(document):
    # 1- tokenization
    tokens = word_tokenize(document)
    # 2- lower case on alpha and leave unchaged others
    ###tokens = [t.lower() if t.isalpha() else t for t in tokens] ## pb sur similarity.max(): à voir
    # 3- remove stopwords and punctuation
    stop_words = set(stopwords.words('english') + list(string.punctuation))
    tokens = [t for t in tokens if not t in stop_words]
    # 4- stemming
    stemmer = PorterStemmer() #build root by removing some known suffix and prefix
    tokens_stem = [stemmer.stem(w) for w in tokens]
    return tokens_stem

In [11]:
## base d'entraînement de vectorizer: Asource + Bsource + Csource + Dsource -> database
def read_data_preproc_dataframe():
    files_ids = ['A','B','C','D']

    # database is a txt file
    # in the same rep as the code to make it easier
    liste_sentences = []
    for id_letter in files_ids:
        database = id_letter + 'source.txt'
        with open(database, 'r') as file:
            data = file.read()
            liste_sentences.append(data)

    # Build the dataframe from the list of sentences
    df = pd.DataFrame(data=liste_sentences)
    df.columns
    df.columns = ['Text']
    #df['Text_preproc'] = df['Text'].apply(preprocessing) ## pas utilisé en fait
    print(df)
    print(f'df.shape: {df.shape}')

    ## TF-IDF entraîné sur tout le corpus
    ## !!! OK avec le preproc de base de sklearn sur col: 'Text' pour la fonction: get_closest_sentence() !!!
    TFIDF_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
    TF_IDF = TFIDF_vectorizer.fit_transform(df.Text).toarray()

    return TFIDF_vectorizer, TF_IDF, df.Text 

In [12]:
TFIDF_vectorizer, TF_IDF, df_Text = read_data_preproc_dataframe()

                                                Text
0  In ages which have no record these islands wer...
1  No oil spill is entirely benign. Depending on ...
2  Descartes has been heralded as the first moder...
3  Normal science, the activity in which most sci...
df.shape: (4, 1)


In [ ]:
## copié de EXO-5
def get_closest_sentence(query, tf_idf, vectorizer, df_text):
    closest_sentence = ""
    query_TFIDF = vectorizer.transform(query).toarray()
    #print(pd.DataFrame(data=query_TFIDF, columns=TFIDF_vectorizer.get_feature_names_out()))

    # cosine similarity between the query and the movies
    similarity = cosine_similarity(query_TFIDF, tf_idf)

    #print(f"Similarity.max(): {similarity.max()}") #MAX = 0.0 dans les tests
    #print(f'similarity.shape: {similarity.shape}')
    #print(f'similarity.argmax(): {similarity.argmax()}')

    closest_sentence = df_text[similarity.argmax()]
    print(f"phrase la + proche de ** {query[0]} **:\n {closest_sentence}")
    return similarity.max(), closest_sentence